<a href="https://colab.research.google.com/github/brunogresende/ppca-aed/blob/main/Aloca%C3%A7%C3%A3o_de_Pessoas_em_Trabalho_Remoto_Scheduling_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
#!pip install -U pulp
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable, LpConstraint
import math

In [85]:
#notations
n     = 120  #number of days for planning (20 = 1 month)
tAJr  = 50  #number of junior analysts
tASr  = 20  #number of senior analysts
tE    = 10  #humber of specialists
max_day_absence = math.floor((tAJr + tASr + tE)*0.3) #max absence allowed in team (30%)

#decision variables
var_LAJr = LpVariable.dicts("AbsenceAJr", (range(n), range(tAJr)), 0, 1, "Binary")
var_LASr = LpVariable.dicts("AbsenceASr", (range(n), range(tASr)), 0, 1, "Binary")
var_LE   = LpVariable.dicts("AbsenceEsp", (range(n), range(tE)), 0, 1, "Binary")

var_AJr = LpVariable.dicts("RemoteAJr", (range(n), range(tAJr)), 0, 1, "Binary")
var_ASr = LpVariable.dicts("RemoteASr", (range(n), range(tASr)), 0, 1, "Binary")
var_E   = LpVariable.dicts("RemoteEsp", (range(n), range(tE)), 0, 1, "Binary")

#objective function
obj = None
for i in range(n):
  for j in range(tE):
    obj += var_E[i][j]
    for jj in range(tAJr):
      obj += var_AJr[i][jj]
      for jjj in range(tASr):
        obj += var_ASr[i][jjj]

problem = LpProblem("Schedule", LpMaximize)
problem += obj

In [86]:
max_day_absence

24

In [87]:
#constraint 0
#absences configuration

#funci AJr 1, 0-5 dias de férias
for i in range(0,5):
  problem += var_LAJr[i][1] == 1

#funci ASr 0, 5-15 dias de férias
for i in range(5,15):
  problem += var_LASr[i][0] == 1

#funci Esp 1, 3 dias de abono
for i in range(17,19):
  problem += var_LE[i][1] == 1

#funci ASr 1, 2 dias de abono
for i in range(4,6):
  problem += var_LASr[i][1] == 1


In [88]:
#constraint 1
#cada funcionário poderá estar trabalhando remoto (1), trabalhando no escritório (0) ou ausente (1), de forma excludente. 

c = None

for i in range(n):
  
  for j in range(tAJr):
    c = lpSum(var_AJr[i][j] + var_LAJr[i][j]) <= 1
    problem += c
    
  for jj in range(tASr):
    c = lpSum(var_ASr[i][jj] + var_LASr[i][jj]) <= 1
    problem += c
      
  for jjj in range(tE):
    c = lpSum(var_E[i][jjj] + var_LE[i][jjj]) <= 1
    problem += c 


In [89]:
#constraint 2 
#cada funcionário poderá ser alocado no máximo 8 dias em home-office no mês. 
for j in range(tAJr):
  c = None
  for i in range(n):
    c += var_AJr[i][j]
  problem += c <=8

for j in range(tASr):
  c = None
  for i in range(n):
    c += var_ASr[i][j]
  problem += c <=8

for j in range(tE):
  c = None
  for i in range(n):
    c += var_E[i][j]
  problem += c <=8


In [90]:
#constraint 3
#a quantidade máxima de ausências do escritório em um dia deve ser no máximo 30% do total de funcionários da equipe. 
c = None
for i in range(n):
  for j in range(tAJr):
    c += var_AJr[i][j] + var_LAJr[i][j]
problem += (c <= max_day_absence*n)

c = None
for i in range(n):
  for j in range(tASr):
    c += var_ASr[i][j] + var_LASr[i][j]
problem += (c <= max_day_absence*n)

c = None
for i in range(n):
  for j in range(tE):
    c += var_E[i][j] + var_LE[i][j]
problem += (c <= max_day_absence*n)

In [91]:
# See the modeled problem
#problem

In [92]:
# Solve the problem
status = problem.solve()

# print results
print(f"Solution Time: {problem.solutionTime}")
print(f"Solution CPU Time: {problem.solutionCpuTime}")
print(f"Status: {problem.status}, {LpStatus[problem.status]}")
print(f"Objective: {problem.objective.value()}")

#for var in problem.variables():
#    print(f"{var.name}: {var.value()}")

#for name, constraint in problem.constraints.items():
#    print(f"{name}: {constraint.value()}")

Solution Time: 0.5555694103240967
Solution CPU Time: 0.5630000000091968
Status: 1, Optimal
Objective: 84080.0
